In [ ]:
# Big thanks to Chen Zecharya, creator of Random Forest code re: Earthquakes on Kaggle for the base ML model.

# Get data from database

In [ ]:
# import datetime
# import sqlalchemy
# from sqlalchemy.ext.automap import automap_base
# from sqlalchemy.orm import Session
# from sqlalchemy import create_engine, inspect, func

# from sqlalchemy import create_engine

In [ ]:
# # user_name_var = input('What is the username of your database? ')
# host_var = input('What is the host address of your database?')
# pw = input("Enter the database password: ")
# # db_name = input("Enter the name of the database: ")

# string_output = "postgresql://root"+':'+pw+'@'+host_var+":5432"+'/postgres'

In [ ]:
# engine = create_engine(string_output)
# connection = engine.connect()
# print("Connected to database...")

In [ ]:
# test_df = pd.read_sql_table(
#     "earthquake_RFmodel_data",
#     con=engine
# )
# test_df

# Run Random Forest Model

In [1]:
# import sklearn and random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
import pandas as pd

## Testing modeling of magnitude with new dataset including nst and rms values.

In [13]:
# pulling test csv into df
test_df = pd.read_csv("./Data_Extraction_transform/DroppedTestwithRMSandNST.csv", index_col = 0)
test_df["unixTime"] = pd.to_datetime(test_df['time']).astype(int) / 10**9
test_nst_df = test_df.loc[test_df["nst"] >= 7]

,time,latitude,longitude,depth,mag,magType,nst,gap,rms,net,id,updated,place,type,magNst,locationSource,unixTime
0,2021-08-19T02:53:29.951Z,38.541200,-119.463900,3.60,1.30,ml,9,58.29,0.1139,nn,nn00818986,2021-08-19T02:56:56.695Z,"3 km NNE of Walker, California",earthquake,4,nn,1.629342e+09
1,2021-08-19T02:49:35.880Z,35.896667,-117.729167,4.48,0.72,ml,9,72.00,0.0900,ci,ci40014776,2021-08-19T02:53:05.273Z,"17km ESE of Little Lake, CA",earthquake,8,ci,1.629341e+09
3,2021-08-19T02:46:16.400Z,38.784668,-122.751000,0.33,1.03,md,13,84.00,0.0100,nc,nc73611791,2021-08-19T02:47:54.191Z,"1km NNE of The Geysers, CA",earthquake,4,nc,1.629341e+09
5,2021-08-19T02:39:51.510Z,34.032167,-116.749500,9.15,2.29,ml,65,44.00,0.1800,ci,ci40014768,2021-08-19T02:50:20.370Z,"13km NNE of Cabazon, CA",earthquake,27,ci,1.629341e+09
6,2021-08-19T02:37:55.960Z,35.456667,-118.314500,3.21,1.45,ml,20,57.00,0.1800,ci,ci40014760,2021-08-19T02:41:43.609Z,"22km SE of Bodfish, CA",earthquake,26,ci,1.629341e+09


In [16]:
# testing magnitude test
independent = ["unixTime", "latitude", "longitude", "nst", "rms"]
dependent = ["mag"]
X = test_nst_df[independent]
y = test_nst_df[dependent]

In [17]:
# setting training and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
import warnings 
warnings.filterwarnings("ignore")
# training and looking for best accuracy
best_score = 0
best_mse = 0
best_model = None

for i in range(1, 50):
    model = RandomForestRegressor(n_estimators = i+1, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    r2 = r2_score(y_test, y_pred)
    score = model.score(X_test, y_test)

    if best_score < score:
        best_score = score
        best_mse = mse
        best_model = model

print(f"Best Model: {best_model}")
print(f"Best MSE: {best_mse}")
print(f"Best Model Score: {best_score}")

Best Model: RandomForestRegressor(n_estimators=48, random_state=42)
Best MSE: 0.06924066300105776
Best Model Score: 0.862718288813799


In [26]:
best_y_pred = best_model.predict(X_test)
predictions = best_y_pred.tolist()
test = y_test["mag"].tolist()

prediction_df = pd.DataFrame({"actual_mag": test, "predicted_mag": predictions}, index=y_test.index)
# prediction_df.head()
# y_test.head()
predictions_id_df = pd.merge(prediction_df, test_nst_df, left_index=True, right_index=True)
predictions_id_df

,actual_mag,predicted_mag,time,latitude,longitude,depth,mag,magType,nst,gap,rms,net,id,updated,place,type,magNst,locationSource,unixTime
6497,1.74,1.745417,2021-07-22T11:01:11.570Z,19.351500,-155.136833,-0.440000,1.74,md,39,110.0,0.14,hv,hv72597887,2021-07-22T22:31:09.890Z,"12 km S of Fern Forest, Hawaii",earthquake,12,hv,1.626952e+09
8186,0.36,0.341458,2021-07-23T14:36:10.620Z,35.884667,-117.674500,4.280000,0.36,ml,11,88.0,0.12,ci,ci39750831,2021-07-23T18:23:16.496Z,"22km ESE of Little Lake, CA",earthquake,4,ci,1.627051e+09
6004,1.25,1.169375,2021-07-23T13:20:54.710Z,38.502500,-119.523167,5.390000,1.25,md,10,77.0,0.05,nc,nc73598156,2021-08-03T20:11:17.553Z,"32km SE of Markleeville, CA",earthquake,7,nc,1.627046e+09
9222,2.30,2.158333,2021-07-21T12:31:01.198Z,31.606910,-103.987350,9.267944,2.30,ml,11,112.0,0.20,tx,tx2021oeev,2021-07-21T16:32:42.527Z,"37 km NNW of Toyah, Texas",earthquake,7,tx,1.626871e+09
17684,1.90,1.925000,2021-07-20T05:40:02.338Z,31.606007,-103.982505,8.908032,1.90,ml,15,114.0,0.20,tx,tx2021obvv,2021-07-20T18:40:56.768Z,"36 km NNW of Toyah, Texas",earthquake,9,tx,1.626760e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084,2.69,2.652917,2021-07-23T20:09:51.770Z,40.319333,-124.369333,28.750000,2.69,md,20,222.0,0.15,nc,nc73598386,2021-08-03T23:51:12.785Z,"7km W of Petrolia, CA",earthquake,20,nc,1.627071e+09
11183,1.82,1.684167,2021-08-02T04:38:20.790Z,38.543000,-119.431000,1.110000,1.82,md,14,76.0,0.10,nc,nc73604191,2021-08-09T22:32:53.439Z,"28km SSW of Smith Valley, NV",earthquake,17,nc,1.627879e+09
2350,2.40,2.170833,2021-08-11T23:29:24.706Z,31.756467,-103.832596,6.157275,2.40,ml,25,48.0,0.20,tx,tx2021prke,2021-08-12T22:45:32.107Z,"22 km WNW of Mentone, Texas",earthquake,10,tx,1.628725e+09
3895,2.39,2.373125,2021-08-08T12:16:19.380Z,19.384000,-155.241833,1.500000,2.39,ml,35,79.0,0.12,hv,hv72630597,2021-08-10T02:28:47.010Z,"6 km S of Volcano, Hawaii",earthquake,9,hv,1.628425e+09


## Testing modeling of depth using dataset with nst and rms values

In [27]:
# testing magnitude test
independent = ["unixTime", "latitude", "longitude", "nst", "rms"]
dependent = ["depth"]
X = test_nst_df[independent]
y = test_nst_df[dependent]

In [28]:
# setting training and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
import warnings 
warnings.filterwarnings("ignore")
# training and looking for best accuracy
best_score = 0
best_mse = 0
best_model = None

for i in range(1, 50):
    model = RandomForestRegressor(n_estimators = i+1, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    r2 = r2_score(y_test, y_pred)
    score = model.score(X_test, y_test)

    if best_score < score:
        best_score = score
        best_mse = mse
        best_model = model

print(f"Best Model: {best_model}")
print(f"Best MSE: {best_mse}")
print(f"Best Model Score: {best_score}")

Best Model: RandomForestRegressor(n_estimators=35, random_state=42)
Best MSE: 18.809780898857564
Best Model Score: 0.816501118005512


In [31]:
best_y_pred = best_model.predict(X_test)
predictions = best_y_pred.tolist()
test = y_test["depth"].tolist()

prediction_df = pd.DataFrame({"actual_depth": test, "predicted_depth": predictions}, index=y_test.index)
prediction_df.head()
predictions_id_df = pd.merge(prediction_df, predictions_id_df, left_index=True, right_index=True)


In [34]:
final_df = predictions_id_df.drop(["actual_depth_y", "predicted_depth_y"], axis=1)

,actual_depth,predicted_depth,Actual Depth,Predicted Depth,actual_mag,predicted_mag,time,latitude,longitude,depth,...,gap,rms,net,id,updated,place,type,magNst,locationSource,unixTime
6497,-0.440000,-0.178000,-0.440000,-0.178000,1.74,1.745417,2021-07-22T11:01:11.570Z,19.351500,-155.136833,-0.440000,...,110.0,0.14,hv,hv72597887,2021-07-22T22:31:09.890Z,"12 km S of Fern Forest, Hawaii",earthquake,12,hv,1.626952e+09
8186,4.280000,5.078857,4.280000,5.078857,0.36,0.341458,2021-07-23T14:36:10.620Z,35.884667,-117.674500,4.280000,...,88.0,0.12,ci,ci39750831,2021-07-23T18:23:16.496Z,"22km ESE of Little Lake, CA",earthquake,4,ci,1.627051e+09
6004,5.390000,5.013714,5.390000,5.013714,1.25,1.169375,2021-07-23T13:20:54.710Z,38.502500,-119.523167,5.390000,...,77.0,0.05,nc,nc73598156,2021-08-03T20:11:17.553Z,"32km SE of Markleeville, CA",earthquake,7,nc,1.627046e+09
9222,9.267944,9.233422,9.267944,9.233422,2.30,2.158333,2021-07-21T12:31:01.198Z,31.606910,-103.987350,9.267944,...,112.0,0.20,tx,tx2021oeev,2021-07-21T16:32:42.527Z,"37 km NNW of Toyah, Texas",earthquake,7,tx,1.626871e+09
17684,8.908032,8.870572,8.908032,8.870572,1.90,1.925000,2021-07-20T05:40:02.338Z,31.606007,-103.982505,8.908032,...,114.0,0.20,tx,tx2021obvv,2021-07-20T18:40:56.768Z,"36 km NNW of Toyah, Texas",earthquake,9,tx,1.626760e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084,28.750000,27.944286,28.750000,27.944286,2.69,2.652917,2021-07-23T20:09:51.770Z,40.319333,-124.369333,28.750000,...,222.0,0.15,nc,nc73598386,2021-08-03T23:51:12.785Z,"7km W of Petrolia, CA",earthquake,20,nc,1.627071e+09
11183,1.110000,1.432286,1.110000,1.432286,1.82,1.684167,2021-08-02T04:38:20.790Z,38.543000,-119.431000,1.110000,...,76.0,0.10,nc,nc73604191,2021-08-09T22:32:53.439Z,"28km SSW of Smith Valley, NV",earthquake,17,nc,1.627879e+09
2350,6.157275,7.200830,6.157275,7.200830,2.40,2.170833,2021-08-11T23:29:24.706Z,31.756467,-103.832596,6.157275,...,48.0,0.20,tx,tx2021prke,2021-08-12T22:45:32.107Z,"22 km WNW of Mentone, Texas",earthquake,10,tx,1.628725e+09
3895,1.500000,1.776857,1.500000,1.776857,2.39,2.373125,2021-08-08T12:16:19.380Z,19.384000,-155.241833,1.500000,...,79.0,0.12,hv,hv72630597,2021-08-10T02:28:47.010Z,"6 km S of Volcano, Hawaii",earthquake,9,hv,1.628425e+09


In [36]:
final_df = final_df.rename(columns={"actual_depth_x": "actual_depth", "predicted_depth_x": "predicted_depth"})

In [38]:
final_df = final_df.drop(["Actual Depth", "Predicted Depth"], axis=1)

,actual_depth,predicted_depth,actual_mag,predicted_mag,time,latitude,longitude,depth,mag,magType,...,gap,rms,net,id,updated,place,type,magNst,locationSource,unixTime
6497,-0.440000,-0.178000,1.74,1.745417,2021-07-22T11:01:11.570Z,19.351500,-155.136833,-0.440000,1.74,md,...,110.0,0.14,hv,hv72597887,2021-07-22T22:31:09.890Z,"12 km S of Fern Forest, Hawaii",earthquake,12,hv,1.626952e+09
8186,4.280000,5.078857,0.36,0.341458,2021-07-23T14:36:10.620Z,35.884667,-117.674500,4.280000,0.36,ml,...,88.0,0.12,ci,ci39750831,2021-07-23T18:23:16.496Z,"22km ESE of Little Lake, CA",earthquake,4,ci,1.627051e+09
6004,5.390000,5.013714,1.25,1.169375,2021-07-23T13:20:54.710Z,38.502500,-119.523167,5.390000,1.25,md,...,77.0,0.05,nc,nc73598156,2021-08-03T20:11:17.553Z,"32km SE of Markleeville, CA",earthquake,7,nc,1.627046e+09
9222,9.267944,9.233422,2.30,2.158333,2021-07-21T12:31:01.198Z,31.606910,-103.987350,9.267944,2.30,ml,...,112.0,0.20,tx,tx2021oeev,2021-07-21T16:32:42.527Z,"37 km NNW of Toyah, Texas",earthquake,7,tx,1.626871e+09
17684,8.908032,8.870572,1.90,1.925000,2021-07-20T05:40:02.338Z,31.606007,-103.982505,8.908032,1.90,ml,...,114.0,0.20,tx,tx2021obvv,2021-07-20T18:40:56.768Z,"36 km NNW of Toyah, Texas",earthquake,9,tx,1.626760e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084,28.750000,27.944286,2.69,2.652917,2021-07-23T20:09:51.770Z,40.319333,-124.369333,28.750000,2.69,md,...,222.0,0.15,nc,nc73598386,2021-08-03T23:51:12.785Z,"7km W of Petrolia, CA",earthquake,20,nc,1.627071e+09
11183,1.110000,1.432286,1.82,1.684167,2021-08-02T04:38:20.790Z,38.543000,-119.431000,1.110000,1.82,md,...,76.0,0.10,nc,nc73604191,2021-08-09T22:32:53.439Z,"28km SSW of Smith Valley, NV",earthquake,17,nc,1.627879e+09
2350,6.157275,7.200830,2.40,2.170833,2021-08-11T23:29:24.706Z,31.756467,-103.832596,6.157275,2.40,ml,...,48.0,0.20,tx,tx2021prke,2021-08-12T22:45:32.107Z,"22 km WNW of Mentone, Texas",earthquake,10,tx,1.628725e+09
3895,1.500000,1.776857,2.39,2.373125,2021-08-08T12:16:19.380Z,19.384000,-155.241833,1.500000,2.39,ml,...,79.0,0.12,hv,hv72630597,2021-08-10T02:28:47.010Z,"6 km S of Volcano, Hawaii",earthquake,9,hv,1.628425e+09


In [40]:
# save to separate csv
final_df.to_csv("./ML_Predictions/predictions_full_data.csv")